# Data processing for the synthetic population of Montréal

In [1]:
import pandas as pd
import numpy as np
from utils import *

In [2]:
def round_to_integer(value, seed=0):
    np.random.seed = seed
    integer_part = int(value)
    diff = abs(value - integer_part)
    if np.random.rand() > diff:
        return integer_part
    else:
        return integer_part + 1

def transpose_df(df, keep_all=False):
    df = df.copy()
    df['variable'] = df['variable'].astype(str)
    df.index = df['variable']
    df.drop(columns=['variable'], inplace=True)
    if keep_all:
        result = df.transpose()
    else:
        result = df[['value']].transpose()
    return result

def find_nearest_zone(data, zone):
    """
    Return the codes of the four nearest dissemination areas to da_code
    """
    neighbors = list(set(data.index.get_level_values(level=0)))
    neighbors.sort()
    index = neighbors.index(zone)
    if index == 0:
        index += 2
    if index == len(neighbors) - 1:
        index -= 2
    neighbors.remove(zone)
    return neighbors[index - 2:index + 2]

def format_variable(variable):
    if variable[1] == 1:
        return [variable[0]]
    else:
        radical = variable[0] + "_"
        return [radical + str(i) for i in range(1, variable[1] + 1)]

def compute_probability_distribution(data, variable):
    variable_cat = format_variable(variable)
    return data.loc[:, variable_cat].div(data.loc[:, variable_cat].sum(axis=1), axis=0)

def compute_average_probability_distribution(data, zone_code, variable):
    """
    Compute the probability distribition of variable X using the 4 nearest zones to zone code
    :param: data: dataframe of data of length > 1
    :params: zone_code: zone geographic code. Example Dissemination area 24663119
    :params: variable: Tuple of variable name and upper bound category. Example ("hh_size", 5).
    return list of probability distribution of the same length as the given upper limit
    """
    neighbors = data.loc[find_nearest_zone(data, zone_code)]
    if isinstance(data.index, pd.MultiIndex):  # DF with multiindex
        level = 1
    else:
        level = None
    return compute_probability_distribution(neighbors, variable).mean(level=level)  # Average probability distribution over the 4 most nearest DAs    

def detect_missing_data(data, variable):
    """
    Detect zones where data are missing on the distribution of HH. These data are coded by StatCan as zeros and no NAs.
    """
    variable_cat = format_variable(variable)
    filter_missing_data = data.loc[:, variable_cat].sum(axis=1) == 0
    return list(data[filter_missing_data].index)

def estimate_number_hh(data, zone_code, pop_col='1', variable=("hh_size", 5)):
    """
    Estimate the number of HH given a total population and a probability distribution of a characteristic of HH, often HH size.
    If X is the number of HH, then the sum of persons in these X HH should equal that of persons given by StatCan (y).
    Put in another form $\sum_{i}^{5}p_i * m_i * X = y$
    Where $p_i$ is the probability of HH size $i$ and $m_i$ is the number of persons per HH. I assume here a limit of 5 persons in HH size 5 (5 persons and plus).
    """
    probability = compute_average_probability_distribution(data, zone_code, variable)
    return round_to_integer(data.loc[zone_code, pop_col] / (probability * range(1, variable[1] + 1)).sum())

def fill_in_missing_data(data, zone, variable, total_population_col="1", hh_variable=("hh_size", 5), hh=True):
    """
    Fill in missing data using the probability distribution of variable X computed from geographically nearest zones
    """
    variable_cat = format_variable(variable)
    probability = compute_average_probability_distribution(data, zone, variable)
    if hh:
        variable_cat_ref = format_variable(hh_variable)
        estimated_number_hh = data.loc[zone, variable_cat_ref].sum()
        if estimated_number_hh == 0:
            estimated_number_hh = estimate_number_hh(data, zone, total_population_col, hh_variable)
    else:
        estimated_number_hh = data.loc[zone, total_population_col]
    hh_distribution = list(probability * estimated_number_hh)
    result = [round_to_integer (x, seed=0) for x in hh_distribution]
    if sum(result) == 0:
        result[np.argmax(hh_distribution)] += estimated_number_hh
    data.loc[zone, variable_cat] = result

def fill_in_dataframe(data, variable, total_population_col="1", hh_variable=("hh_size", 5), hh=True):
    zones_with_missing_data = detect_missing_data(data, variable)
    [fill_in_missing_data(data, zone, variable, total_population_col, hh_variable, hh) for zone in zones_with_missing_data]

def check_for_data_integrity(data, variable, ref_variable=("hh_size", 5), tolerance=30):
    """
    Check if all HH characteristics produce the same total number of HH
    """
    ref_variable_cat = format_variable(ref_variable)
    variable_cat = format_variable(variable)
    abs_max_difference = abs((data.loc[:, ref_variable_cat].sum(axis=1) - data.loc[:, variable_cat].sum(axis=1))).max()
    return abs_max_difference < tolerance

def construct_list(position, size):
    result = np.zeros(size)
    result[position] = 1
    return result

def find_maximum_column(df, variable):
    """
    Return the position of the maximum of the variable
    """
    variable_cat = format_variable(variable)
    return np.argmax(np.array(df.loc[:, variable_cat]), axis=1)

def mock_probability(li, size=5):
    """
    Process list "li" to produce a dataframe of shape len(li) * size. 
    The dataframe contains zeros except position indicated in li which are filled with ones
    """
    assert max(li) < size
    return pd.DataFrame([construct_list(i, size) for i in li])

def correct_data(data, variable, ref_var=("hh_size", 5), tol=30):
    """
    The total number of HH computed using each HH characteristics should be equal. This function assures that this is
    true for each Dissemination area (DA)
    This function assumes that one characteristic of HH is a reference variable that all other variables should clone
    """
    data = data.copy()
    ref_variable_cat = format_variable(ref_var)
    variable_cat = format_variable(variable)
    if not check_for_data_integrity(data, variable, ref_var, tol):
        difference = data.loc[:, ref_variable_cat].sum(axis=1) - data.loc[:, variable_cat].sum(axis=1)
        data.loc[:, variable_cat] += compute_probability_distribution(data, variable).mul(difference, axis=0).applymap(lambda x: round_to_integer(x, seed=0))
        difference = data.loc[:, ref_variable_cat].sum(axis=1) - data.loc[:, variable_cat].sum(axis=1)
        new_probability = mock_probability(find_maximum_column(data, variable), variable[1])
        new_probability.index = data.index
        new_probability.columns = data.loc[:, variable_cat].columns
        data.loc[:, variable_cat] += new_probability.mul(difference, axis=0)
        data[data <  0] = 0
    assert (data.loc[:, variable_cat] >= 0).all().all()
    return data

In [3]:
idx = pd.IndexSlice

[Import md here]

## Data sources

https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/download-telecharger/comp/page_dl-tc.cfm?Lang=E

Graphic here

## Households(HH)
+ HH size
+ HH type (not available in the OD of Montreal)
+ HH income
+ HH number of cars (not available in census data). SAAQ provides data on car fleet with no association to persons or HH

### Household size
- 1 person (standard column name in the census data 52)
- 2 persons (53)
- 3 persons (54)
- 4 persons (55)
- 5 persons and more (56)

### Household type (Sample data do not include clear information on households types, we shall use a simple HH type based on the presence of children in the HH. The presence of kids has an impact on travel behaviour of some members of the HH)
- Presence of children: yes
- Presence of children: no

### Household income

Different income sources are included in the census: Employment income, before/after tax income, etc. I use the HH total income (lines 760 to 779). To match this income with the sample income variable, one should check the definition of the  sample income.
 
Household income ($)
- \< 30,000
- 30,000 - 59,999
- 60,000 - 89,999
- 90,000 - 149,999
- \> 150,000

Census data must be grouped to match the income brackets above

+ 760.   Under \$5,000
+ 761.   5,000 to \$9,999
+ 762.   10,000 to \$14,999
+ 763.   15,000 to \$19,999
+ 764.   20,000 to \$24,999
+ 765.   25,000 to \$29,999
+ 766.   30,000 to \$34,999
+ 767.   35,000 to \$39,999
+ 768.   40,000 to \$44,999
+ 769.   45,000 to \$49,999
+ 770.   50,000 to \$59,999
+ 771.   60,000 to \$69,999
+ 772.   70,000 to \$79,999
+ 773.   80,000 to \$89,999
+ 774.   90,000 to \$99,999
+ 776.   100,000 to \$124,999
+ 777.   125,000 to \$149,999
+ 778.   150,000 to \$199,999
+ 779.   200,000 and over

# Households

## Data processing for households (HH) at the Census Tract level (CT)

### Select the rows needed for the population generation

In [4]:
col_hh_size = list(range(52, 57))  # Information about the size of the HH is contained in rows between 52 and 57.
col_hh_income = list(range(760, 780))
col_hh_income.remove(775)  # 775 is the sum of 776 and forth
col_hh_type = [92, 95, 96]

selection_columns = col_hh_size + col_hh_income + col_hh_type

In [4]:
selection_columns = list(range(1900, 1920))
selection_columns += [1867]

### Income processing

+ hh_income_1: < 30,000 \$
+ hh_income_2: 30,000 - 59,999
+ hh_income_3: 60,000 - 89,999
+ hh_income_4: 90,000 - 149,999
+ hh_income_5: > 150,000

### HH size

- hh_size_1: 1 person (52)
- hh_size_2: 2 persons (53)
- hh_size_3: 3 persons (54)
- hh_size_4: 4 persons (55)
- hh_size_5: 5 persons and more (56)

### Household type (not used in the case of Montreal since this information is not available at the sample (OD) level)
The OD survey of Montreal does not include information on blood relations between members of the HH. The only information one can infer is related to HH with kids (presence of kids) or not.

- HH with kids (type 1)
- HH without kids (type 2)

__/!\ It is not possible to compute the number of HH with kids. The most close number to that is to use HH types and to assume that a portion of Multitple-census families have kids. This portion is not known from the 
census but can be infered from the PUMF.__

## Data processing of HH at the Dissemination Area (DA) level

In [6]:
da_data = pd.read_csv("../data/census/DA/census_da_cma_montreal_all_data.csv")

### This is a huge dataset. It duplicates the 6469 DA that form the CMA of Montreal 2247 times, i.e. each DA has 2247 records as in the case of CMA

In [32]:
selection_conditions = da_data["Member ID: Profile of Dissemination Areas (2247)"].isin(selection_columns + [1])  # [1] is the name of the total population column 

In [33]:
montreal_da_data = da_data[selection_conditions]

In [34]:
montreal_da_data = montreal_da_data[['GEO_CODE (POR)', 'Dim: Sex (3): Member ID: [1]: Total - Sex',
                                     'Member ID: Profile of Dissemination Areas (2247)']].copy()

In [35]:
montreal_da_data.rename(
    columns={
        'GEO_CODE (POR)': 'geo',
        'Member ID: Profile of Dissemination Areas (2247)': 'variable',
        'Dim: Sex (3): Member ID: [1]: Total - Sex': 'value'},
    inplace=True)

In [36]:
montreal_da_data

,geo,value,variable
0,24520101,787,1
1866,24520101,415,1867
1899,24520101,0,1900
1900,24520101,10,1901
1901,24520101,0,1902
...,...,...,...
14535510,24760051,50,1915
14535511,24760051,10,1916
14535512,24760051,45,1917
14535513,24760051,25,1918


# /!\

__ASSUMPTION 1__: For some DA, StatCan mentions that data is unreliable (F) or simply omits records for confidentiality reasons. I replace these missing data with NA and infer them later from the distribution of the data

In [37]:
montreal_da_data.replace(['F', 'x', '..'], np.nan, inplace=True)

In [38]:
montreal_da_data['value'] = montreal_da_data[~montreal_da_data['value'].isna()]['value'].astype(int)

### Check results

In [441]:
assert len(montreal_da_data) == (len(col_hh_size) + len(col_hh_income) + len(col_hh_type) + 1) * 6469  # Select for each DA a number of records contained in col_hh_XX

In [40]:
montreal_da_data = montreal_da_data.groupby(['geo']).apply(transpose_df)

In [41]:
montreal_da_data.index = montreal_da_data.index.levels[0]

### Process income data

In [16]:
montreal_da_data['hh_income_1'] = montreal_da_data.loc[:, '760':'765'].sum(axis=1)
montreal_da_data['hh_income_2'] = montreal_da_data.loc[:, '766':'770'].sum(axis=1)
montreal_da_data['hh_income_3'] = montreal_da_data.loc[:, '771':'773'].sum(axis=1)
montreal_da_data['hh_income_4'] = montreal_da_data.loc[:, '774':'777'].sum(axis=1)
montreal_da_data['hh_income_5'] = montreal_da_data.loc[:, '778':'779'].sum(axis=1)

In [17]:
montreal_da_data.drop(columns=[str(c) for c in col_hh_income], inplace=True)  # Drop original data used for the income

### Process size data

In [42]:
hh_immi_dict = {'1141': 'p_non_imm', '1142': 'p_imm', '1150': 'p_non_res'}
montreal_da_data.rename(columns=hh_immi_dict, inplace=True)

In [44]:
hh_tra_dict = {'1867': 'trav', '1868': 'non_trav', '1869': 'non_appli'}
montreal_da_data.rename(columns=hh_tra_dict, inplace=True)

In [45]:
hh_min_dict = {'1325': 'asie_sud', '1326': 'chinois', '1327': 'noir', '1328': 'filip', '1329': 'amer_lat', '1330': 'arab', '1331': 'asie_sudest',
                '1332': 'asie_ouest', '1333': 'koree', '1334': 'japon', '1335': 'eni', '1336': 'mni'}
montreal_da_data.rename(columns=hh_min_dict, inplace=True)

In [18]:
hh_size_dict = {'52': 'hh_size_1', '53': 'hh_size_2', '54': 'hh_size_3', '55': 'hh_size_4', '56': 'hh_size_5'}
montreal_da_data.rename(columns=hh_size_dict, inplace=True)

In [42]:
montreal_da_data

variable,1,1867,1900,1901,1902,1903,1904,1905,1906,1907,...,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919
geo,,,,,,,,,,,,,,,,,,,,,
24520101,787.0,415.0,0.0,10.0,0.0,65.0,70.0,10.0,60.0,20.0,...,15.0,15.0,0.0,10.0,25.0,40.0,10.0,20.0,20.0,25.0
24520102,589.0,325.0,0.0,0.0,0.0,45.0,45.0,30.0,55.0,25.0,...,10.0,10.0,0.0,20.0,15.0,65.0,0.0,10.0,10.0,10.0
24520103,476.0,230.0,0.0,0.0,0.0,25.0,25.0,10.0,30.0,10.0,...,0.0,10.0,0.0,0.0,25.0,45.0,0.0,10.0,25.0,10.0
24520104,900.0,495.0,15.0,0.0,0.0,70.0,65.0,20.0,70.0,40.0,...,0.0,25.0,0.0,15.0,25.0,95.0,0.0,30.0,35.0,15.0
24520105,539.0,285.0,0.0,0.0,0.0,45.0,25.0,0.0,40.0,25.0,...,0.0,10.0,0.0,0.0,30.0,60.0,0.0,25.0,35.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24750245,981.0,500.0,0.0,10.0,0.0,90.0,95.0,45.0,65.0,25.0,...,10.0,20.0,0.0,15.0,35.0,35.0,0.0,15.0,25.0,10.0
24750247,2076.0,1120.0,10.0,0.0,10.0,100.0,225.0,55.0,140.0,55.0,...,15.0,40.0,0.0,70.0,110.0,125.0,15.0,60.0,35.0,65.0
24750248,3957.0,2170.0,10.0,10.0,35.0,300.0,340.0,60.0,220.0,125.0,...,30.0,110.0,0.0,90.0,130.0,315.0,60.0,85.0,120.0,135.0


### Processing HH type data

# /!\ I assume that in all DAs 30% of multiple-family households contain at least one kid. This information (30%) come from the PUMF at the regional level

In [48]:
montreal_da_data['96'] *= 0.3
montreal_da_data['96'] = montreal_da_data[~montreal_da_data['96'].isna()]['96'].astype(int)
montreal_da_data['hh_type_1'] = montreal_da_data['95'] + montreal_da_data['96']  # The sum of one-census familis with children plus a portion of multiple-census families that is assumed to have kids
montreal_da_data['hh_type_2'] = montreal_da_data['92'] - montreal_da_data['hh_type_1']

montreal_da_data.drop(columns=[str(c) for c in col_hh_type], inplace=True)

In [30]:
montreal_da_data

variable,1,1900,1901,1902,1903,1904,1905,1906,1907,1908,...,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919
geo,,,,,,,,,,,,,,,,,,,,,
24520101,787.0,0.0,10.0,0.0,65.0,70.0,10.0,60.0,20.0,10.0,...,15.0,15.0,0.0,10.0,25.0,40.0,10.0,20.0,20.0,25.0
24520102,589.0,0.0,0.0,0.0,45.0,45.0,30.0,55.0,25.0,0.0,...,10.0,10.0,0.0,20.0,15.0,65.0,0.0,10.0,10.0,10.0
24520103,476.0,0.0,0.0,0.0,25.0,25.0,10.0,30.0,10.0,0.0,...,0.0,10.0,0.0,0.0,25.0,45.0,0.0,10.0,25.0,10.0
24520104,900.0,15.0,0.0,0.0,70.0,65.0,20.0,70.0,40.0,0.0,...,0.0,25.0,0.0,15.0,25.0,95.0,0.0,30.0,35.0,15.0
24520105,539.0,0.0,0.0,0.0,45.0,25.0,0.0,40.0,25.0,0.0,...,0.0,10.0,0.0,0.0,30.0,60.0,0.0,25.0,35.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24750245,981.0,0.0,10.0,0.0,90.0,95.0,45.0,65.0,25.0,10.0,...,10.0,20.0,0.0,15.0,35.0,35.0,0.0,15.0,25.0,10.0
24750247,2076.0,10.0,0.0,10.0,100.0,225.0,55.0,140.0,55.0,10.0,...,15.0,40.0,0.0,70.0,110.0,125.0,15.0,60.0,35.0,65.0
24750248,3957.0,10.0,10.0,35.0,300.0,340.0,60.0,220.0,125.0,25.0,...,30.0,110.0,0.0,90.0,130.0,315.0,60.0,85.0,120.0,135.0


In [50]:
montreal_da_data.to_csv("../cma_montreal_inequities.csv")

### Checking data

In [450]:
montreal_da_data[montreal_da_data['hh_size_1'].isna()]

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,
24560169,5.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24560260,130.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24560296,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24560334,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24580677,20.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24650595,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24661208,240.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24661299,894.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
24661729,353.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [451]:
montreal_da_data = montreal_da_data[~((montreal_da_data['1'] == 0) | (montreal_da_data['1'].isna()))]
montreal_da_data.fillna(0, inplace = True)

/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/pandas/core/frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [452]:
fill_in_dataframe(montreal_da_data, ("hh_size", 5), hh_variable=("hh_size", 5))
fill_in_dataframe(montreal_da_data, ("hh_income", 5), hh_variable=("hh_size", 5))
fill_in_dataframe(montreal_da_data, ("hh_type", 2), hh_variable=("hh_size", 5))

/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [453]:
montreal_da_data = correct_data(montreal_da_data, ("hh_income", 5), ref_var=("hh_size", 5), tol=5)
montreal_da_data = correct_data(montreal_da_data, ("hh_type", 2), ref_var=("hh_size", 5), tol=5)

In [454]:
ref = montreal_da_data.loc[:, 'hh_size_1':'hh_size_5'].sum(axis=1)

In [455]:
(ref - montreal_da_data.loc[:, 'hh_type_1': 'hh_type_2'].sum(axis=1)).sort_values(ascending=True)

geo
24520101    0.0
24662228    0.0
24662227    0.0
24662226    0.0
24662225    0.0
           ... 
24650577    0.0
24650576    0.0
24650575    0.0
24650586    0.0
24760051    0.0
Length: 6442, dtype: float64

In [456]:
(ref - montreal_da_data.loc[:, 'hh_income_1': 'hh_income_5'].sum(axis=1)).sort_values(ascending=True)

geo
24520101    0.0
24662228    0.0
24662227    0.0
24662226    0.0
24662225    0.0
           ... 
24650577    0.0
24650576    0.0
24650575    0.0
24650586    0.0
24760051    0.0
Length: 6442, dtype: float64

### Check for further missing data

For some DAs, StatCan gives only the total population but no information on the distribution of HH according to size, income or type. These data are replaced by zero values and are not noticeable when controlling or NAs as I have done before.

In [457]:
montreal_da_data.loc[montreal_da_data.loc[:, "hh_size_1":"hh_size_5"].sum(axis=1) == 0]

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,


In [458]:
montreal_da_data.loc[montreal_da_data.loc[:, "hh_income_1":"hh_income_5"].sum(axis=1) == 0]

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,


In [459]:
montreal_da_data.loc[montreal_da_data.loc[:, "hh_type_1":"hh_type_2"].sum(axis=1) == 0]

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,


# Persons

+ Age: see categories below
+ Sex: female and male
+ Occupation: working, not working, or not applicable
+ Driving licence: yes,no, not applicable
+ Place of work: from home, ouside home, not applicable

### Age (columns from 10 to 29)
+ 1 : 0 to 4 ans
+ 2 : 5 to 9 ans
+ 3 : 10 to 14 ans
+ 4 : 15 to 19 ans
+ 5 : 20 to 24 ans
+ 6 : 25 to 29 ans
+ 7 : 30 to 34 ans
+ 8 : 35 to 39 ans
+ 9 : 40 to 44 ans
+ 10 : 45 to 49 ans
+ 11 : 50 to 54 ans
+ 12 : 55 to 59 ans
+ 13 : 60 to 64 ans
+ 14 : 65 to 69 ans
+ 15 : 70 to 74 ans
+ 16 : 75 ans et +

### Sex
+ 1 : Male
+ 2 : Female

### Occupation
+ 1: Not working (retired persons and unemployed, students with part-time jobs and over 15 y/o, deny answer, home) (1874)
+ 2: Full-time job (1876)
+ 3: Part-time job (1877)
+ 4: Not applicable (people under the age of 15) (infered from age categories)

mapping Occupation variable with HTS data:

1. Travailleur à temps complet -> category 2
2. Travailleur à temps partiel -> 3
3. Étudiant / élève: 
    + if age < 15 -> 4
    + if age $\geq$ 15 -> 1
4. Retraité -> 1
5. Autre -> 1
6. N/A : enfant de 4 ans et moins -> 4
7. À la maison -> 1
8. Refus -> 1

### Driving licence (SAAQ data)
+ 1 : Yes
+ 2 : No
+ 5 : Not applicable (age < 16)

### Place of work
+ From home (1921)
+ Outside home (1922, 1923, 1924)
+ Not applicable (the rest: under the age of 15)

## Data processing of persons at the regional level

In [460]:
occupation_dict = {'1874': 'occupation_1', '1876': 'occupation_2', '1877': 'occupation_3'}

# Person characteristics at the Dissemination Area

In [461]:
col_p_age = list(range(10, 30))
col_p_age.remove(13)  # this is a sum row
col_p_age.remove(24)  # this is a sum row
col_p_occupation = [1874, 1876, 1877]
col_p_workplace = [1921]

selection_columns_p = col_p_age + col_p_workplace

In [462]:
k = [str(k) for k in col_p_age]
v = ['age_{}'.format(i) for i in range(1, 16)]
age_dict = dict(zip(k, v))

In [463]:
p_selection = da_data["Member ID: Profile of Dissemination Areas (2247)"].isin([1] + selection_columns_p)  # [1] is for the total population

In [464]:
montreal_da_p_data = da_data[p_selection]

In [465]:
montreal_da_p_data = montreal_da_p_data[['GEO_CODE (POR)', 'Member ID: Profile of Dissemination Areas (2247)',
                                         'Dim: Sex (3): Member ID: [1]: Total - Sex', 'Dim: Sex (3): Member ID: [2]: Male', 'Dim: Sex (3): Member ID: [3]: Female']].copy()

In [466]:
montreal_da_p_data

,GEO_CODE (POR),Member ID: Profile of Dissemination Areas (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,24520101,1,787,...,...
9,24520101,10,40,20,15
10,24520101,11,45,20,20
11,24520101,12,40,25,15
13,24520101,14,50,25,25
...,...,...,...,...,...
14533621,24760051,26,55,35,25
14533622,24760051,27,30,15,20
14533623,24760051,28,25,15,15
14533624,24760051,29,10,5,5


In [467]:
montreal_da_p_data.rename(
    columns={
        'GEO_CODE (POR)': 'geo',
        'Member ID: Profile of Dissemination Areas (2247)': 'variable',
        'Dim: Sex (3): Member ID: [1]: Total - Sex': 'value',
        'Dim: Sex (3): Member ID: [2]: Male': 'Male', 
        'Dim: Sex (3): Member ID: [3]: Female': 'Female',
        '1': 'Pop'},
    inplace=True)

# /!\

__ASSUMPTION 1__: For some DA, StatCan mentions that data is unreliable (F) or simply omits records for confidentiality reasons. I replace these missing data with NA and infer them later from the distribution of the data

In [468]:
montreal_da_p_data.replace(['F', 'x', '..', '...'], np.nan, inplace=True)

In [469]:
montreal_da_p_data['value'] = montreal_da_p_data[~montreal_da_p_data['value'].isna()]['value'].astype(int)
montreal_da_p_data['Male'] = montreal_da_p_data[~montreal_da_p_data['Male'].isna()]['Male'].astype(int)
montreal_da_p_data['Female'] = montreal_da_p_data[~montreal_da_p_data['Female'].isna()]['Female'].astype(int)

### Check results

In [470]:
assert len(montreal_da_p_data) == (len(col_p_age) + len(col_p_workplace) + 1) * 6469  # Select for each DA a number of records contained in col_hh_XX

In [471]:
montreal_da_p_data = montreal_da_p_data.groupby(['geo']).apply(lambda x: transpose_df(x, keep_all=True))

In [472]:
montreal_da_p_data = montreal_da_p_data.loc[idx[:, ['value', 'Male', 'Female']], :]

#### Age

In [473]:
montreal_da_p_data.rename(columns=age_dict, inplace=True)

In [474]:
montreal_da_p_data.loc[idx[:, :], 'age_16'] = montreal_da_p_data.loc[idx[:, :], '27':'29'].sum(axis=1)

In [475]:
montreal_da_p_data.drop(columns=['27', '28', '29'], inplace=True)

### Total population

In [476]:
montreal_da_p_data.rename(columns={"1": "Pop"}, inplace=True)

#### Sex

In [477]:
montreal_da_p_data

variable            Pop  age_1  age_2  age_3  age_4  age_5  age_6  age_7  \
geo                                                                        
24520101 value    787.0   40.0   45.0   40.0   50.0   45.0   40.0   40.0   
         Male       NaN   20.0   20.0   25.0   25.0   30.0   20.0   30.0   
         Female     NaN   15.0   20.0   15.0   25.0   20.0   20.0   15.0   
24520102 value    589.0   25.0   50.0   45.0   40.0   35.0   30.0   30.0   
         Male       NaN   10.0   30.0   30.0   25.0   20.0   20.0   15.0   
...                 ...    ...    ...    ...    ...    ...    ...    ...   
24760050 Male       NaN   30.0   15.0   20.0   15.0   20.0   15.0   20.0   
         Female     NaN   25.0   20.0   10.0   20.0   20.0   10.0   25.0   
24760051 value   1009.0   40.0   30.0   35.0   35.0   35.0   45.0   55.0   
         Male       NaN   20.0   10.0   15.0   20.0   20.0   25.0   30.0   
         Female     NaN   15.0   15.0   15.0   15.0   15.0   15.0   30.0   

variable         age_8  age_9  age_10  age_11  age_12  age_13  age_14  age_15  \
geo                                                                             
24520101 value    50.0   45.0    50.0    90.0    60.0    75.0    50.0    35.0   
         Male     25.0   20.0    25.0    40.0    30.0    45.0    20.0    15.0   
         Female   25.0   20.0    25.0    45.0    30.0    30.0    30.0    15.0   
24520102 value    45.0   45.0    45.0    60.0    45.0    40.0    15.0    15.0   
         Male     25.0   25.0    20.0    30.0    25.0    20.0    10.0     5.0   
...                ...    ...     ...     ...     ...     ...     ...     ...   
24760050 Male     30.0   20.0    35.0    45.0    50.0    65.0    40.0    25.0   
         Female   15.0   25.0    20.0    40.0    70.0    45.0    30.0    25.0   
24760051 value    50.0   50.0    80.0    95.0   120.0   115.0    95.0    55.0   
         Male     30.0   30.0    45.0    55.0    55.0    70.0    55.0    35.0   
         Female   20.0   20.0    35.0    50.0    70.0    45.0    40.0    25.0   

variable         1921  age_16  
geo                            
24520101 value   35.0    50.0  
         Male    20.0    20.0  
         Female  15.0    30.0  
24520102 value   45.0    20.0  
         Male    10.0    10.0  
...               ...     ...  
24760050 Male    20.0    25.0  
         Female  25.0    20.0  
24760051 value   15.0    65.0  
         Male    15.0    35.0  
         Female   0.0    40.0  

[19407 rows x 18 columns]

In [478]:
montreal_da_p_data.loc[idx[:, 'Male'], 'sex_1'] = montreal_da_p_data.loc[idx[:, 'Male'], idx[format_variable(('age', 16))]].sum(axis=1)
montreal_da_p_data.loc[idx[:, 'value'], 'sex_1'] = list(montreal_da_p_data.loc[idx[:, 'Male'], 'sex_1'])

montreal_da_p_data.loc[idx[:, 'Female'], 'sex_2'] = montreal_da_p_data.loc[idx[:, 'Female'], idx[format_variable(('age', 16))]].sum(axis=1)
montreal_da_p_data.loc[idx[:, 'value'], 'sex_2'] = list(montreal_da_p_data.loc[idx[:, 'Female'], 'sex_2'])

### Workplace

In [479]:
montreal_da_p_data.rename(columns={'1921': 'workplace_1'}, inplace=True)  # Work from home
montreal_da_p_data['workplace_2'] = montreal_da_p_data.loc[idx[:, 'value'], ['sex_1', 'sex_2']].sum(1) - montreal_da_p_data['workplace_1']  # Work outside home

### Check data for NAs and Missing values

In [480]:
montreal_da_p_data

variable            Pop  age_1  age_2  age_3  age_4  age_5  age_6  age_7  \
geo                                                                        
24520101 value    787.0   40.0   45.0   40.0   50.0   45.0   40.0   40.0   
         Male       NaN   20.0   20.0   25.0   25.0   30.0   20.0   30.0   
         Female     NaN   15.0   20.0   15.0   25.0   20.0   20.0   15.0   
24520102 value    589.0   25.0   50.0   45.0   40.0   35.0   30.0   30.0   
         Male       NaN   10.0   30.0   30.0   25.0   20.0   20.0   15.0   
...                 ...    ...    ...    ...    ...    ...    ...    ...   
24760050 Male       NaN   30.0   15.0   20.0   15.0   20.0   15.0   20.0   
         Female     NaN   25.0   20.0   10.0   20.0   20.0   10.0   25.0   
24760051 value   1009.0   40.0   30.0   35.0   35.0   35.0   45.0   55.0   
         Male       NaN   20.0   10.0   15.0   20.0   20.0   25.0   30.0   
         Female     NaN   15.0   15.0   15.0   15.0   15.0   15.0   30.0   

variable         age_8  age_9  ...  age_11  age_12  age_13  age_14  age_15  \
geo                            ...                                           
24520101 value    50.0   45.0  ...    90.0    60.0    75.0    50.0    35.0   
         Male     25.0   20.0  ...    40.0    30.0    45.0    20.0    15.0   
         Female   25.0   20.0  ...    45.0    30.0    30.0    30.0    15.0   
24520102 value    45.0   45.0  ...    60.0    45.0    40.0    15.0    15.0   
         Male     25.0   25.0  ...    30.0    25.0    20.0    10.0     5.0   
...                ...    ...  ...     ...     ...     ...     ...     ...   
24760050 Male     30.0   20.0  ...    45.0    50.0    65.0    40.0    25.0   
         Female   15.0   25.0  ...    40.0    70.0    45.0    30.0    25.0   
24760051 value    50.0   50.0  ...    95.0   120.0   115.0    95.0    55.0   
         Male     30.0   30.0  ...    55.0    55.0    70.0    55.0    35.0   
         Female   20.0   20.0  ...    50.0    70.0    45.0    40.0    25.0   

variable         workplace_1  age_16  sex_1  sex_2  workplace_2  
geo                                                              
24520101 value          35.0    50.0  410.0  380.0        755.0  
         Male           20.0    20.0  410.0    NaN          NaN  
         Female         15.0    30.0    NaN  380.0          NaN  
24520102 value          45.0    20.0  320.0  280.0        555.0  
         Male           10.0    10.0  320.0    NaN          NaN  
...                      ...     ...    ...    ...          ...  
24760050 Male           20.0    25.0  470.0    NaN          NaN  
         Female         25.0    20.0    NaN  420.0          NaN  
24760051 value          15.0    65.0  550.0  465.0       1000.0  
         Male           15.0    35.0  550.0    NaN          NaN  
         Female          0.0    40.0    NaN  465.0          NaN  

[19407 rows x 21 columns]

In [481]:
montreal_da_p_data = montreal_da_p_data[['Pop'] + format_variable(("age", 16)) + format_variable(("sex", 2)) + format_variable(("workplace", 2))]

In [482]:
test = montreal_da_p_data.reset_index()
mask = test.isna()

In [483]:
idx_to_drop = test[(test.level_1 == 'value') & ((test.Pop == 0) | (test['Pop'].isna()))].geo

In [484]:
montreal_da_p_data.drop(idx_to_drop, level=0, inplace=True)  # Drop the above mentioned DAs from dataframe

/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [485]:
test = montreal_da_p_data.reset_index()
mask = test.isna()

In [486]:
idx_missing_data = test[(mask.age_1)].geo.unique()

In [487]:
montreal_da_p_data_new = montreal_da_p_data.loc[idx[:,'value'], :].droplevel(1)

In [488]:
montreal_da_p_data_new[montreal_da_p_data_new.index.isin(idx_missing_data)]

variable,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,age_11,age_12,age_13,age_14,age_15,age_16,sex_1,sex_2,workplace_1,workplace_2
geo,,,,,,,,,,,,,,,,,,,,,
24560169,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24560260,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24580677,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24661208,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24661299,894.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,10.0,-10.0
24661729,353.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
24662998,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24663370,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN
24663381,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN


In [489]:
fill_in_dataframe(montreal_da_p_data_new, ("sex", 2), 'Pop', hh=False)

In [490]:
fill_in_dataframe(montreal_da_p_data_new, ("age", 16), 'Pop', hh=False)

In [491]:
fill_in_dataframe(montreal_da_p_data_new, ("workplace", 2), 'Pop', hh=False)

In [492]:
montreal_da_p_data_new.loc[detect_missing_data(montreal_da_p_data_new, ("age", 16)), ['Pop'] + format_variable(("age", 16))]

variable,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10,age_11,age_12,age_13,age_14,age_15,age_16
geo,,,,,,,,,,,,,,,,,


In [493]:
montreal_da_p_data_new = correct_data(montreal_da_p_data_new, ("age", 16), ref_var=("Pop", 1))
montreal_da_p_data_new = correct_data(montreal_da_p_data_new, ("workplace", 2), ref_var=("Pop", 1))
montreal_da_p_data_new = correct_data(montreal_da_p_data_new, ("sex", 2), ref_var=("Pop", 1))

In [494]:
(montreal_da_p_data_new['Pop'] - montreal_da_p_data_new[format_variable(("age", 16))].sum(1)).sort_values()

geo
24663381   -2.0
24663382   -1.0
24663396   -1.0
24560169   -1.0
24520101    0.0
           ... 
24650577    0.0
24650576    0.0
24650575    0.0
24650586    0.0
24760051    0.0
Length: 6442, dtype: float64

In [495]:
(montreal_da_p_data_new['Pop'] - montreal_da_p_data_new[format_variable(("sex", 2))].sum(1)).sort_values()

geo
24520101    0.0
24662228    0.0
24662227    0.0
24662226    0.0
24662225    0.0
           ... 
24650577    0.0
24650576    0.0
24650575    0.0
24650586    0.0
24760051    0.0
Length: 6442, dtype: float64

In [496]:
(montreal_da_p_data_new['Pop'] - montreal_da_p_data_new[format_variable(("workplace", 2))].sum(1)).sort_values()

geo
24520101    0.0
24662228    0.0
24662227    0.0
24662226    0.0
24662225    0.0
           ... 
24650577    0.0
24650576    0.0
24650575    0.0
24650586    0.0
24760051    0.0
Length: 6442, dtype: float64

In [497]:
montreal_da_p_data_new

variable,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,age_11,age_12,age_13,age_14,age_15,age_16,sex_1,sex_2,workplace_1,workplace_2
geo,,,,,,,,,,,,,,,,,,,,,
24520101,787.0,41.0,44.0,41.0,49.0,44.0,41.0,41.0,49.0,44.0,...,76.0,59.0,75.0,49.0,36.0,49.0,407.0,380.0,36.0,751.0
24520102,589.0,25.0,51.0,46.0,40.0,35.0,31.0,30.0,45.0,45.0,...,61.0,45.0,40.0,15.0,15.0,20.0,314.0,275.0,46.0,543.0
24520103,476.0,36.0,36.0,25.0,45.0,26.0,20.0,35.0,40.0,40.0,...,41.0,40.0,25.0,21.0,5.0,5.0,229.0,247.0,0.0,476.0
24520104,900.0,50.0,71.0,80.0,76.0,51.0,45.0,60.0,76.0,80.0,...,76.0,55.0,25.0,35.0,15.0,25.0,479.0,421.0,35.0,865.0
24520105,539.0,30.0,31.0,25.0,31.0,36.0,45.0,36.0,41.0,25.0,...,40.0,66.0,40.0,31.0,21.0,20.0,283.0,256.0,35.0,504.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24750245,981.0,76.0,86.0,61.0,55.0,31.0,51.0,86.0,96.0,71.0,...,91.0,71.0,70.0,40.0,30.0,15.0,525.0,456.0,25.0,956.0
24750247,2076.0,136.0,176.0,146.0,146.0,110.0,105.0,131.0,181.0,186.0,...,190.0,146.0,100.0,66.0,51.0,25.0,1091.0,985.0,70.0,2006.0
24750248,3957.0,415.0,395.0,299.0,191.0,166.0,249.0,409.0,415.0,309.0,...,244.0,224.0,186.0,105.0,61.0,60.0,2032.0,1925.0,180.0,3777.0


# Control variables at the Census Tract level

In [5]:
cd_data = pd.read_csv("../data/census/CT/98-401-X2016043_English_CSV_data.csv")

/home/omanout/.local/share/virtualenvs/playground/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
cd_data = cd_data[(cd_data.GEO_LEVEL == 2) & (cd_data['GEO_CODE (POR)'].apply(lambda x: str(x)[:3] == '462'))].copy()

In [26]:
selection_conditions = cd_data["Member ID: Profile of Census Tracts (2247)"].isin(selection_columns)

In [27]:
montreal_cd_data = cd_data[selection_conditions]

In [28]:
montreal_cd_data

,CENSUS_YEAR,GEO_CODE (POR),GEO_LEVEL,GEO_NAME,GNR,GNR_LF,DATA_QUALITY_FLAG,ALT_GEO_CODE,DIM: Profile of Census Tracts (2247),Member ID: Profile of Census Tracts (2247),Notes: Profile of Census Tracts (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
1410735,2016,4620001.0,2,0001.00,3.6,3.1,0,462000100,Employed,1867,NaN,1435,710,725
1410768,2016,4620001.0,2,0001.00,3.6,3.1,0,462000100,"11 Agriculture, forestry, fishing and hunting",1900,NaN,0,10,0
1410769,2016,4620001.0,2,0001.00,3.6,3.1,0,462000100,"21 Mining, quarrying, and oil and gas extraction",1901,NaN,0,0,0
1410770,2016,4620001.0,2,0001.00,3.6,3.1,0,462000100,22 Utilities,1902,NaN,0,10,0
1410771,2016,4620001.0,2,0001.00,3.6,3.1,0,462000100,23 Construction,1903,NaN,75,75,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588126,2016,4622402.0,2,2402.00,2.5,5.2,0,462240200,62 Health care and social assistance,1915,NaN,260,25,235
3588127,2016,4622402.0,2,2402.00,2.5,5.2,0,462240200,"71 Arts, entertainment and recreation",1916,NaN,55,30,25
3588128,2016,4622402.0,2,2402.00,2.5,5.2,0,462240200,72 Accommodation and food services,1917,NaN,115,55,60
3588129,2016,4622402.0,2,2402.00,2.5,5.2,0,462240200,81 Other services (except public administration),1918,NaN,40,25,10


In [29]:
montreal_cd_data = montreal_cd_data[['GEO_CODE (POR)', 'Dim: Sex (3): Member ID: [2]: Male', 'Dim: Sex (3): Member ID: [3]: Female', 'Member ID: Profile of Census Tracts (2247)']].copy()

In [30]:
montreal_cd_data.rename(
    columns={
        'GEO_CODE (POR)': 'geo',
        'Member ID: Profile of Census Tracts (2247)': 'variable',
        'Dim: Sex (3): Member ID: [2]: Male': 'male_value', 
        'Dim: Sex (3): Member ID: [3]: Female': 'female_value'},
    inplace=True)

In [31]:
montreal_cd_data

,geo,male_value,female_value,variable
1410735,4620001.0,710,725,1867
1410768,4620001.0,10,0,1900
1410769,4620001.0,0,0,1901
1410770,4620001.0,10,0,1902
1410771,4620001.0,75,0,1903
...,...,...,...,...
3588126,4622402.0,25,235,1915
3588127,4622402.0,30,25,1916
3588128,4622402.0,55,60,1917
3588129,4622402.0,25,10,1918


In [32]:
montreal_cd_data.replace(['F', 'x', '..', '...'], np.nan, inplace=True)

In [33]:
montreal_cd_data['male_value'] = montreal_cd_data[~montreal_cd_data['male_value'].isna()]['male_value'].astype(float)
montreal_cd_data['female_value'] = montreal_cd_data[~montreal_cd_data['female_value'].isna()]['female_value'].astype(float)

In [39]:
montreal_cd_data

variable                  1867  1900  1901  1902   1903   1904  1905   1906  \
geo                                                                           
4620001.0 male_value     710.0  10.0   0.0  10.0   75.0   95.0  35.0  105.0   
          female_value   725.0   0.0   0.0   0.0    0.0   40.0  15.0  125.0   
4620002.0 male_value     870.0   0.0   0.0  20.0   35.0  145.0  35.0  115.0   
          female_value   825.0   0.0   0.0  10.0   15.0   40.0  25.0  125.0   
4620003.0 male_value    1475.0   0.0   0.0   0.0  120.0  245.0  85.0  160.0   
...                        ...   ...   ...   ...    ...    ...   ...    ...   
4622400.0 female_value   600.0  15.0   0.0   0.0    0.0   40.0  35.0  105.0   
4622401.0 male_value    1050.0  25.0   0.0   0.0  250.0  235.0  30.0  155.0   
          female_value   975.0  20.0   0.0   0.0   15.0   60.0  30.0  175.0   
4622402.0 male_value     930.0  15.0   0.0  15.0   90.0  170.0  25.0  100.0   
          female_value   900.0  15.0   0.0  10.0   30.0   45.0  10.0   90.0   

variable                 1907  1908  ...  1910   1911  1912   1913   1914  \
geo                                  ...                                    
4620001.0 male_value     55.0  25.0  ...  10.0   50.0   0.0   40.0   45.0   
          female_value   10.0  20.0  ...   0.0   25.0   0.0    0.0  100.0   
4620002.0 male_value     60.0  30.0  ...  20.0   90.0   0.0   40.0   35.0   
          female_value   25.0  30.0  ...  10.0   30.0   0.0   35.0   85.0   
4620003.0 male_value    155.0  50.0  ...  15.0  100.0   0.0  155.0   60.0   
...                       ...   ...  ...   ...    ...   ...    ...    ...   
4622400.0 female_value   25.0  10.0  ...   0.0   35.0   0.0   20.0   50.0   
4622401.0 male_value     60.0  15.0  ...  15.0   55.0   0.0   35.0   30.0   
          female_value   15.0  10.0  ...  15.0   50.0   0.0   10.0  110.0   
4622402.0 male_value     55.0  30.0  ...  15.0   85.0   0.0   15.0   45.0   
          female_value   15.0  25.0  ...  10.0   85.0   0.0   20.0   90.0   

variable                 1915  1916  1917  1918   1919  
geo                                                     
4620001.0 male_value     30.0  20.0  35.0  40.0   55.0  
          female_value  180.0  25.0  80.0  55.0   35.0  
4620002.0 male_value     70.0  15.0  55.0  40.0   65.0  
          female_value  205.0  20.0  80.0  35.0   40.0  
4620003.0 male_value     90.0  45.0  75.0  65.0  105.0  
...                       ...   ...   ...   ...    ...  
4622400.0 female_value  150.0  10.0  45.0  30.0   10.0  
4622401.0 male_value     50.0   0.0  25.0  40.0   60.0  
          female_value  270.0  20.0  55.0  55.0   40.0  
4622402.0 male_value     25.0  30.0  55.0  25.0  130.0  
          female_value  235.0  25.0  60.0  10.0   90.0  

[1940 rows x 21 columns]

In [35]:
def transpose_df(df, keep_all=True):
    df = df.copy()
    df['variable'] = df['variable'].astype(str)
    df.index = df['variable']
    df.drop(columns=['variable'], inplace=True)
    if keep_all:
        result = df.transpose()
    else:
        result = df[['value']].transpose()
    return result

In [36]:
montreal_cd_data = montreal_cd_data.groupby(['geo']).apply(transpose_df)

In [37]:
montreal_cd_data = montreal_cd_data.loc[idx[:, ['male_value', 'female_value']], :].copy()

In [24]:
montreal_cd_data = montreal_cd_data.droplevel(level=1)

In [38]:
montreal_cd_data.to_csv("synpop_data/NAICS2012_job_classification_census_tract_by_sex.csv")

### Process income data

In [62]:
montreal_cd_data['hh_income_1'] = montreal_cd_data.loc[:, '760':'765'].sum(axis=1)
montreal_cd_data['hh_income_2'] = montreal_cd_data.loc[:, '766':'770'].sum(axis=1)
montreal_cd_data['hh_income_3'] = montreal_cd_data.loc[:, '771':'773'].sum(axis=1)
montreal_cd_data['hh_income_4'] = montreal_cd_data.loc[:, '774':'777'].sum(axis=1)
montreal_cd_data['hh_income_5'] = montreal_cd_data.loc[:, '778':'779'].sum(axis=1)

In [63]:
montreal_cd_data.drop(columns=[str(c) for c in col_hh_income], inplace=True)  # Drop original data used for the income

### Process size data

In [64]:
hh_size_dict = {'52': 'hh_size_1', '53': 'hh_size_2', '54': 'hh_size_3', '55': 'hh_size_4', '56': 'hh_size_5'}
montreal_cd_data.rename(columns=hh_size_dict, inplace=True)

In [66]:
montreal_cd_data.rename(columns=hh_tra_dict, inplace=True)
montreal_cd_data.rename(columns=hh_immi_dict, inplace=True)
montreal_cd_data.rename(columns=hh_min_dict, inplace=True)

### Processing HH type data

In [65]:
montreal_cd_data['96'] *= 0.3  # 30% of multiple-family HHs are assumed to have kids. This share has been computed using PUMF Hierarachical data
montreal_cd_data['96'] = montreal_cd_data[~montreal_cd_data['96'].isna()]['96'].astype(int)
montreal_cd_data['hh_type_1'] = montreal_cd_data['95'] + montreal_cd_data['96']  # The sum of one-census families with children plus a portion of multiple-census families that is assumed to have kids
montreal_cd_data['hh_type_2'] = montreal_cd_data['92'] - montreal_cd_data['hh_type_1']

montreal_cd_data.drop(columns=[str(c) for c in col_hh_type], inplace=True)

In [67]:
montreal_cd_data

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,p_non_imm,p_imm,p_non_res,asie_sud,...,1917,1918,1919,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,,,,,,,,,
4620001.0,2638.0,590.0,445.0,145.0,105.0,40.0,2285.0,405.0,10.0,15.0,...,115.0,90.0,95.0,305.0,450.0,320.0,210.0,35.0,378.0,947.0
4620002.0,3516.0,795.0,540.0,205.0,135.0,80.0,2880.0,435.0,20.0,10.0,...,130.0,75.0,115.0,495.0,595.0,350.0,255.0,65.0,544.0,1216.0
4620003.0,6373.0,1115.0,935.0,440.0,300.0,150.0,4780.0,1430.0,40.0,15.0,...,140.0,150.0,230.0,725.0,1005.0,585.0,500.0,125.0,1057.0,1878.0
4620004.0,3176.0,705.0,530.0,200.0,120.0,40.0,2700.0,400.0,40.0,20.0,...,130.0,85.0,100.0,420.0,560.0,305.0,240.0,75.0,469.0,1136.0
4620005.0,3060.0,740.0,520.0,170.0,110.0,50.0,2620.0,395.0,20.0,0.0,...,95.0,95.0,85.0,480.0,525.0,310.0,230.0,55.0,418.0,1177.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622303.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4622304.0,1094.0,100.0,185.0,60.0,60.0,35.0,1065.0,30.0,0.0,0.0,...,15.0,10.0,45.0,45.0,110.0,95.0,120.0,65.0,166.0,274.0
4622400.0,2729.0,265.0,475.0,170.0,140.0,80.0,2665.0,60.0,0.0,0.0,...,65.0,60.0,55.0,175.0,350.0,270.0,265.0,65.0,418.0,717.0


In [68]:
montreal_cd_data.to_csv("../cma_montreal_CT_inequities.csv")

### Checking data

In [514]:
montreal_cd_data[montreal_cd_data.isna().any(axis=1)]

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,
4620014.02,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620040.00,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620071.00,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620091.00,5.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620094.02,5.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620127.02,5.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620145.00,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620189.00,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4620229.00,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [515]:
montreal_cd_data = montreal_cd_data[~((montreal_cd_data['1'].isna()) | (montreal_cd_data['1'] == 0))]

In [516]:
fill_in_dataframe(montreal_cd_data, ("hh_size", 5), hh_variable=("hh_size", 5))
fill_in_dataframe(montreal_cd_data, ("hh_income", 5), hh_variable=("hh_size", 5))
fill_in_dataframe(montreal_cd_data, ("hh_type", 2), hh_variable=("hh_size", 5))

/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/omanout@forcity.local/.local/share/virtualenvs/synthpop/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [517]:
montreal_cd_data = correct_data(montreal_cd_data, ("hh_income", 5), ref_var=("hh_size", 5), tol=5)
montreal_cd_data = correct_data(montreal_cd_data, ("hh_type", 2), ref_var=("hh_size", 5), tol=5)

In [518]:
ref = montreal_cd_data.loc[:,'hh_size_1': 'hh_size_5'].sum(axis=1)

In [519]:
(ref - montreal_cd_data.loc[:, 'hh_type_1': 'hh_type_2'].sum(axis=1)).sort_values(ascending=True)

geo
4620001.00    0.0
4620684.10    0.0
4620684.11    0.0
4620685.02    0.0
4620685.03    0.0
             ... 
4620306.00    0.0
4620307.00    0.0
4620308.00    0.0
4620290.09    0.0
4622402.00    0.0
Length: 959, dtype: float64

In [520]:
(ref - montreal_cd_data.loc[:,'hh_income_1': 'hh_income_5'].sum(axis=1)).sort_values(ascending=True)

geo
4620001.00    0.0
4620684.10    0.0
4620684.11    0.0
4620685.02    0.0
4620685.03    0.0
             ... 
4620306.00    0.0
4620307.00    0.0
4620308.00    0.0
4620290.09    0.0
4622402.00    0.0
Length: 959, dtype: float64

In [521]:
montreal_cd_data

variable,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2
geo,,,,,,,,,,,,,
4620001.0,2638.0,590.0,445.0,145.0,105.0,40.0,306.0,452.0,321.0,211.0,35.0,378.0,947.0
4620002.0,3516.0,795.0,540.0,205.0,135.0,80.0,494.0,589.0,351.0,256.0,65.0,543.0,1212.0
4620003.0,6373.0,1115.0,935.0,440.0,300.0,150.0,725.0,1005.0,585.0,500.0,125.0,1059.0,1881.0
4620004.0,3176.0,705.0,530.0,200.0,120.0,40.0,420.0,553.0,306.0,241.0,75.0,468.0,1127.0
4620005.0,3060.0,740.0,520.0,170.0,110.0,50.0,477.0,519.0,310.0,229.0,55.0,417.0,1173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622302.0,2688.0,365.0,505.0,160.0,110.0,55.0,215.0,445.0,290.0,195.0,50.0,403.0,792.0
4622304.0,1094.0,100.0,185.0,60.0,60.0,35.0,46.0,111.0,97.0,120.0,66.0,166.0,274.0
4622400.0,2729.0,265.0,475.0,170.0,140.0,80.0,175.0,352.0,271.0,266.0,66.0,418.0,712.0


# Persons at the CT

In [522]:
p_selection = cd_data["Member ID: Profile of Census Tracts (2247)"].isin([1] + selection_columns_p)

In [523]:
montreal_cd_p_data = cd_data[p_selection]

In [524]:
montreal_cd_p_data = montreal_cd_p_data[['GEO_CODE (POR)', 'Member ID: Profile of Census Tracts (2247)',
                                         'Dim: Sex (3): Member ID: [1]: Total - Sex', 'Dim: Sex (3): Member ID: [2]: Male',
                                         'Dim: Sex (3): Member ID: [3]: Female']].copy()

In [525]:
montreal_cd_p_data

,GEO_CODE (POR),Member ID: Profile of Census Tracts (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
1408869,4620001.0,1,2638,...,...
1408878,4620001.0,10,130,80,50
1408879,4620001.0,11,105,65,40
1408880,4620001.0,12,95,50,45
1408882,4620001.0,14,120,55,65
...,...,...,...,...,...
3586237,4622402.0,26,75,40,40
3586238,4622402.0,27,50,20,25
3586239,4622402.0,28,40,25,15
3586240,4622402.0,29,35,20,15


In [526]:
montreal_cd_p_data.rename(
    columns={
        'GEO_CODE (POR)': 'geo',
        'Member ID: Profile of Census Tracts (2247)': 'variable',
        'Dim: Sex (3): Member ID: [1]: Total - Sex': 'value',
        'Dim: Sex (3): Member ID: [2]: Male': 'Male', 
        'Dim: Sex (3): Member ID: [3]: Female': 'Female',
        '1': 'Pop'},
    inplace=True)

# /!\

__ASSUMPTION 1__: For some DA, StatCan mentions that data is unreliable (F) or simply omits records for confidentiality reasons. I replace these missing data with NA and infer them later from the distribution of the data

In [527]:
montreal_cd_p_data.replace(['F', 'x', '..', '...'], np.nan, inplace=True)

In [528]:
montreal_cd_p_data['value'] = montreal_cd_p_data[~montreal_cd_p_data['value'].isna()]['value'].astype(float)
montreal_cd_p_data['Male'] = montreal_cd_p_data[~montreal_cd_p_data['Male'].isna()]['Male'].astype(float)
montreal_cd_p_data['Female'] = montreal_cd_p_data[~montreal_cd_p_data['Female'].isna()]['Female'].astype(float)

In [529]:
montreal_cd_p_data = montreal_cd_p_data.groupby(['geo']).apply(lambda x: transpose_df(x, keep_all=True))

In [530]:
montreal_cd_p_data = montreal_cd_p_data.loc[idx[:, ['value', 'Male', 'Female']], :].copy()

#### Age

In [531]:
montreal_cd_p_data.rename(columns=age_dict, inplace=True)

In [532]:
montreal_cd_p_data.loc[idx[:, :], 'age_16'] = montreal_cd_p_data.loc[idx[:, :], '27':'29'].sum(axis=1)

In [533]:
montreal_cd_p_data.drop(columns=['27', '28', '29'], inplace=True)

### Total population

In [534]:
montreal_cd_p_data.rename(columns={"1": "Pop"}, inplace=True)

#### Sex

In [535]:
montreal_cd_p_data.loc[idx[:, 'Male'], 'sex_1'] = montreal_cd_p_data.loc[idx[:, 'Male'], idx[format_variable(('age', 16))]].sum(axis=1)
montreal_cd_p_data.loc[idx[:, 'value'], 'sex_1'] = list(montreal_cd_p_data.loc[idx[:, 'Male'], 'sex_1'])

montreal_cd_p_data.loc[idx[:, 'Female'], 'sex_2'] = montreal_cd_p_data.loc[idx[:, 'Female'], idx[format_variable(('age', 16))]].sum(axis=1)
montreal_cd_p_data.loc[idx[:, 'value'], 'sex_2'] = list(montreal_cd_p_data.loc[idx[:, 'Female'], 'sex_2'])

### Workplace

In [536]:
montreal_cd_p_data.rename(columns={'1921': 'workplace_1'}, inplace=True)
montreal_cd_p_data['workplace_2'] = montreal_cd_p_data.loc[idx[:, 'value'], ['sex_1', 'sex_2']].sum(1) - montreal_cd_p_data['workplace_1']

In [537]:
montreal_cd_p_data

variable             Pop  age_1  age_2  age_3  age_4  age_5  age_6  age_7  \
geo                                                                         
4620001.0 value   2638.0  130.0  105.0   95.0  120.0  160.0  200.0  210.0   
          Male       NaN   80.0   65.0   50.0   55.0   80.0   85.0  100.0   
          Female     NaN   50.0   40.0   45.0   65.0   85.0  115.0  110.0   
4620002.0 value   3516.0  235.0  185.0  135.0  145.0  230.0  255.0  275.0   
          Male       NaN  110.0   90.0   60.0   70.0  110.0  125.0  140.0   
...                  ...    ...    ...    ...    ...    ...    ...    ...   
4622401.0 Male       NaN  125.0  170.0   95.0  110.0   90.0  105.0  145.0   
          Female     NaN  145.0  145.0  105.0  105.0   80.0  120.0  155.0   
4622402.0 value   3484.0  325.0  360.0  255.0  195.0  150.0  180.0  275.0   
          Male       NaN  165.0  165.0  120.0  105.0   70.0   80.0  130.0   
          Female     NaN  160.0  200.0  130.0   95.0   75.0   95.0  140.0   

variable          age_8  age_9  ...  age_11  age_12  age_13  age_14  age_15  \
geo                             ...                                           
4620001.0 value   170.0  160.0  ...   210.0   275.0   180.0   155.0   105.0   
          Male     85.0  100.0  ...   125.0   140.0    85.0    75.0    65.0   
          Female   80.0   60.0  ...    90.0   135.0   100.0    80.0    40.0   
4620002.0 value   255.0  235.0  ...   280.0   255.0   225.0   175.0   145.0   
          Male    125.0  115.0  ...   145.0   110.0   110.0    75.0    65.0   
...                 ...    ...  ...     ...     ...     ...     ...     ...   
4622401.0 Male    145.0  125.0  ...   130.0   155.0   115.0    85.0    80.0   
          Female  140.0  120.0  ...   150.0   130.0   110.0    85.0    65.0   
4622402.0 value   355.0  270.0  ...   185.0   195.0   180.0   140.0    75.0   
          Male    175.0  140.0  ...    95.0    90.0    90.0    75.0    40.0   
          Female  175.0  130.0  ...    85.0   100.0    90.0    60.0    40.0   

variable          workplace_1  age_16   sex_1   sex_2  workplace_2  
geo                                                                 
4620001.0 value          60.0   160.0  1345.0  1295.0       2580.0  
          Male           30.0    60.0  1345.0     NaN          NaN  
          Female         30.0    95.0     NaN  1295.0          NaN  
4620002.0 value          40.0   260.0  1660.0  1840.0       3460.0  
          Male           30.0    85.0  1660.0     NaN          NaN  
...                       ...     ...     ...     ...          ...  
4622401.0 Male           85.0    55.0  1840.0     NaN          NaN  
          Female         60.0    65.0     NaN  1840.0          NaN  
4622402.0 value         110.0   125.0  1715.0  1745.0       3350.0  
          Male           55.0    65.0  1715.0     NaN          NaN  
          Female         55.0    55.0     NaN  1745.0          NaN  

[2910 rows x 21 columns]

### Check data for NAs and Missing values

In [538]:
test = montreal_cd_p_data.reset_index()
mask = test.isna()

In [539]:
idx_to_drop = test[(test.level_1 == 'value') & ((test.Pop == 0) | (test.Pop.isna()))].geo

In [540]:
idx_to_drop

45      4620014.02
120     4620040.00
228     4620071.00
477     4620145.00
618     4620189.00
750     4620229.00
870     4620268.03
2157    4620732.03
2442    4620832.00
2820    4622006.00
2895    4622303.00
Name: geo, dtype: float64

In [541]:
montreal_cd_p_data.drop(idx_to_drop, level=0, inplace=True)  # Drop the above mentioned DAs from dataframe

In [542]:
test = montreal_cd_p_data.reset_index()
mask = test.isna()

In [543]:
idx_missing_data = test[mask['age_1']].geo.unique()

In [544]:
idx_missing_data

array([4620091.  , 4620094.02, 4620127.02, 4620315.  , 4620440.  ,
       4620887.07, 4622003.  , 4622010.  ])

In [545]:
montreal_cd_p_data_new = montreal_cd_p_data.loc[idx[:,'value'], :].droplevel(1)

In [546]:
montreal_cd_p_data_new.loc[idx_missing_data]

variable,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,age_11,age_12,age_13,age_14,age_15,workplace_1,age_16,sex_1,sex_2,workplace_2
geo,,,,,,,,,,,,,,,,,,,,,
4620091.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4620094.02,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4620127.02,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4620315.00,240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4620440.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4620887.07,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4622003.00,130.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
4622010.00,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN


In [547]:
fill_in_dataframe(montreal_cd_p_data_new, ("age", 16), "Pop", hh=False)
fill_in_dataframe(montreal_cd_p_data_new, ("workplace", 2), "Pop", hh=False)
fill_in_dataframe(montreal_cd_p_data_new, ("sex", 2), "Pop", hh=False)

In [548]:
montreal_cd_p_data_new = correct_data(montreal_cd_p_data_new, ("age", 16), ref_var=("Pop", 1), tol=2)

In [549]:
montreal_cd_p_data_new = correct_data(montreal_cd_p_data_new, ("workplace", 2), ref_var=("Pop", 1), tol=2)
montreal_cd_p_data_new = correct_data(montreal_cd_p_data_new, ("sex", 2), ref_var=("Pop", 1), tol=2)

In [550]:
ref = montreal_cd_p_data_new['Pop']

In [551]:
(ref - montreal_cd_p_data_new[format_variable(("age", 16))].sum(1)).sort_values(ascending=True)

geo
4620094.02   -1.0
4620001.00    0.0
4620684.11    0.0
4620685.02    0.0
4620685.03    0.0
             ... 
4620307.00    0.0
4620308.00    0.0
4620309.00    0.0
4620291.01    0.0
4622402.00    0.0
Length: 959, dtype: float64

In [552]:
(ref - montreal_cd_p_data_new[format_variable(("workplace", 2))].sum(1)).sort_values(ascending=True)

geo
4620001.00    0.0
4620684.10    0.0
4620684.11    0.0
4620685.02    0.0
4620685.03    0.0
             ... 
4620306.00    0.0
4620307.00    0.0
4620308.00    0.0
4620290.09    0.0
4622402.00    0.0
Length: 959, dtype: float64

In [553]:
(ref - montreal_cd_p_data_new[format_variable(("sex", 2))].sum(1)).sort_values(ascending=True)

geo
4620001.00    0.0
4620684.10    0.0
4620684.11    0.0
4620685.02    0.0
4620685.03    0.0
             ... 
4620306.00    0.0
4620307.00    0.0
4620308.00    0.0
4620290.09    0.0
4622402.00    0.0
Length: 959, dtype: float64

In [554]:
montreal_cd_p_data_new = montreal_cd_p_data_new[['Pop'] + format_variable(("age", 16)) + format_variable(("sex", 2)) + format_variable(("workplace", 2))]

In [555]:
montreal_cd_p_data_new

variable,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,age_11,age_12,age_13,age_14,age_15,age_16,sex_1,sex_2,workplace_1,workplace_2
geo,,,,,,,,,,,,,,,,,,,,,
4620001.0,2638.0,131.0,105.0,95.0,120.0,160.0,200.0,211.0,170.0,160.0,...,210.0,271.0,180.0,155.0,105.0,160.0,1342.0,1296.0,60.0,2578.0
4620002.0,3516.0,235.0,185.0,135.0,146.0,230.0,256.0,275.0,255.0,236.0,...,266.0,255.0,226.0,175.0,145.0,260.0,1667.0,1849.0,40.0,3476.0
4620003.0,6373.0,330.0,330.0,330.0,305.0,350.0,430.0,465.0,455.0,385.0,...,510.0,518.0,485.0,345.0,275.0,445.0,3096.0,3277.0,120.0,6253.0
4620004.0,3176.0,140.0,141.0,111.0,146.0,206.0,221.0,230.0,205.0,211.0,...,241.0,291.0,241.0,180.0,140.0,261.0,1550.0,1626.0,40.0,3136.0
4620005.0,3060.0,171.0,126.0,110.0,110.0,191.0,236.0,201.0,225.0,195.0,...,260.0,268.0,235.0,190.0,141.0,206.0,1515.0,1545.0,75.0,2985.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622302.0,2688.0,125.0,130.0,126.0,145.0,171.0,181.0,136.0,165.0,130.0,...,190.0,215.0,235.0,165.0,130.0,283.0,1317.0,1371.0,56.0,2632.0
4622304.0,1094.0,41.0,61.0,61.0,71.0,61.0,41.0,46.0,51.0,66.0,...,91.0,100.0,92.0,87.0,56.0,97.0,548.0,546.0,30.0,1064.0
4622400.0,2729.0,165.0,140.0,135.0,135.0,150.0,145.0,160.0,141.0,160.0,...,241.0,261.0,250.0,201.0,145.0,145.0,1443.0,1286.0,110.0,2619.0


In [556]:
del da_data, cd_data

## Reindex dataframes

In [557]:
census_tract_reindex = dict(zip(montreal_cd_data.index, range(1, len(montreal_cd_data) + 1)))
dissemination_areas_reindex = dict(zip(montreal_da_data.index, range(1, len(montreal_da_data) + 1)))

In [558]:
montreal_cd_data.reset_index(inplace=True)
montreal_da_data.reset_index(inplace=True)

montreal_cd_p_data_new.reset_index(inplace=True)
montreal_da_p_data_new.reset_index(inplace=True)

In [559]:
montreal_cd_data.index += 1
montreal_da_data.index += 1

montreal_cd_p_data_new.index += 1
montreal_da_p_data_new.index += 1

In [560]:
ct = montreal_cd_data[['geo']].drop_duplicates() * 1000
da = montreal_da_data[['geo']].drop_duplicates()

ct['geo'] = ct['geo'].astype(int)
ct.reset_index(inplace=True)
da.reset_index(inplace=True)

In [561]:
mapping_ct_da = pd.read_csv("../data/shp/matching_DA_montreal.csv", usecols=["DAUID", "CTUID"])

mapping_ct_da["CTUID"] *= 1000
mapping_ct_da['CTUID'] = mapping_ct_da['CTUID'].astype(int)

In [562]:
mapping_ct_da = mapping_ct_da.merge(ct, left_on="CTUID", right_on="geo").rename(columns={"index": "new_ct"})
mapping_ct_da = mapping_ct_da.merge(da, left_on="DAUID", right_on="geo").rename(columns={"index": "new_da"})

In [563]:
mapping_ct_da.drop(columns=['geo_x', 'geo_y'], inplace=True)

# Output writing 

path = 'V_2'

montreal_cd_data.to_csv("{}/region_household_marginals.csv".format(path))
montreal_cd_p_data_new.to_csv("{}/region_person_marginals.csv".format(path))
montreal_da_data.to_csv("{}/household_marginals.csv".format(path))
montreal_da_p_data_new.to_csv("{}/person_marginals.csv".format(path))

mapping_ct_da.to_csv("{}/matching_da_ct.csv".format(path), index=False)

# SAAQ data

I use SAAQ data on driving license and car ownerships to compute marginals for the population at the dissemination area level of resolution. These data have been made available by Jérôme Laviolette (Catherine's team). Jérôme got the data from SAAQ via simple email request.

In [564]:
def compute_age_category(age):
    if age > 80:
        result = 16
    else:
        result = age // 5 + 1
    return result

## Driving license

In [565]:
saaq_data = pd.read_csv("../data/quebec_data/hh_car_license.csv")

In [566]:
saaq_data

,DAUID,CTUID,new_ct,hh_driving_license_1,hh_driving_license_2,hh_driving_license_3,hh_driving_license_4,hh_car_1,hh_car_2,hh_car_3,hh_car_4,hh_car
0,24520101,4620690000,657,85.0,176.0,30.0,20.0,135.0,113.0,38.0,14.0,538.0
1,24520102,4620691000,658,69.0,121.0,24.0,9.0,80.0,99.0,24.0,9.0,389.0
2,24520103,4620691000,658,42.0,100.0,24.0,6.0,52.0,83.0,23.0,11.0,340.0
3,24520104,4620691000,658,68.0,159.0,51.0,23.0,88.0,131.0,43.0,30.0,611.0
4,24520105,4620691000,658,41.0,110.0,28.0,10.0,70.0,78.0,26.0,11.0,356.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6436,24750245,4620740030,724,110.0,208.0,37.0,7.0,123.0,165.0,44.0,13.0,642.0
6437,24750247,4620740030,724,198.0,431.0,96.0,40.0,259.0,336.0,100.0,44.0,1419.0
6438,24750248,4620740040,725,380.0,852.0,125.0,58.0,480.0,661.0,152.0,67.0,2545.0
6439,24760050,4620740010,723,407.0,133.0,15.0,2.0,306.0,145.0,27.0,10.0,721.0


## Join HHs Census and SAAD data

In [567]:
hh = montreal_da_data
hh.rename(columns={"geo": "DAUID"}, inplace=True)
hh["geo"] = hh.index

In [568]:
hh

variable,DAUID,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,hh_income_4,hh_income_5,hh_type_1,hh_type_2,geo
1,24520101,787.0,65.0,135.0,45.0,45.0,25.0,31.0,82.0,83.0,83.0,36.0,123.0,192.0,1
2,24520102,589.0,35.0,70.0,40.0,50.0,15.0,11.0,56.0,56.0,73.0,14.0,108.0,102.0,2
3,24520103,476.0,10.0,60.0,40.0,35.0,15.0,5.0,36.0,47.0,67.0,5.0,95.0,65.0,3
4,24520104,900.0,45.0,85.0,70.0,65.0,40.0,16.0,51.0,98.0,114.0,26.0,179.0,126.0,4
5,24520105,539.0,35.0,80.0,35.0,30.0,20.0,24.0,48.0,59.0,58.0,11.0,91.0,109.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6438,24750245,981.0,80.0,130.0,75.0,45.0,40.0,44.0,93.0,88.0,105.0,40.0,171.0,199.0,6438
6439,24750247,2076.0,95.0,265.0,135.0,160.0,75.0,46.0,154.0,174.0,255.0,101.0,383.0,347.0,6439
6440,24750248,3957.0,240.0,395.0,265.0,325.0,155.0,101.0,278.0,293.0,505.0,203.0,746.0,634.0,6440
6441,24760050,895.0,135.0,175.0,40.0,45.0,20.0,81.0,100.0,94.0,94.0,46.0,103.0,312.0,6441


In [569]:
hh_saaq = pd.merge(hh, saaq_data, on='DAUID', how='left')

In [570]:
hh_saaq

,DAUID,1,hh_size_1,hh_size_2,hh_size_3,hh_size_4,hh_size_5,hh_income_1,hh_income_2,hh_income_3,...,new_ct,hh_driving_license_1,hh_driving_license_2,hh_driving_license_3,hh_driving_license_4,hh_car_1,hh_car_2,hh_car_3,hh_car_4,hh_car
0,24520101,787.0,65.0,135.0,45.0,45.0,25.0,31.0,82.0,83.0,...,657.0,85.0,176.0,30.0,20.0,135.0,113.0,38.0,14.0,538.0
1,24520102,589.0,35.0,70.0,40.0,50.0,15.0,11.0,56.0,56.0,...,658.0,69.0,121.0,24.0,9.0,80.0,99.0,24.0,9.0,389.0
2,24520103,476.0,10.0,60.0,40.0,35.0,15.0,5.0,36.0,47.0,...,658.0,42.0,100.0,24.0,6.0,52.0,83.0,23.0,11.0,340.0
3,24520104,900.0,45.0,85.0,70.0,65.0,40.0,16.0,51.0,98.0,...,658.0,68.0,159.0,51.0,23.0,88.0,131.0,43.0,30.0,611.0
4,24520105,539.0,35.0,80.0,35.0,30.0,20.0,24.0,48.0,59.0,...,658.0,41.0,110.0,28.0,10.0,70.0,78.0,26.0,11.0,356.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6437,24750245,981.0,80.0,130.0,75.0,45.0,40.0,44.0,93.0,88.0,...,724.0,110.0,208.0,37.0,7.0,123.0,165.0,44.0,13.0,642.0
6438,24750247,2076.0,95.0,265.0,135.0,160.0,75.0,46.0,154.0,174.0,...,724.0,198.0,431.0,96.0,40.0,259.0,336.0,100.0,44.0,1419.0
6439,24750248,3957.0,240.0,395.0,265.0,325.0,155.0,101.0,278.0,293.0,...,725.0,380.0,852.0,125.0,58.0,480.0,661.0,152.0,67.0,2545.0
6440,24760050,895.0,135.0,175.0,40.0,45.0,20.0,81.0,100.0,94.0,...,723.0,407.0,133.0,15.0,2.0,306.0,145.0,27.0,10.0,721.0


In [571]:
saaq_error_driving = (hh_saaq.loc[:, format_variable(("hh_size", 5))].sum(1) - hh_saaq.loc[:, format_variable(("hh_driving_license", 4))].sum(1)) < 0

In [572]:
saaq_error_car = (hh_saaq.loc[:, format_variable(("hh_size", 5))].sum(1) - hh_saaq.loc[:, format_variable(("hh_car", 4))].sum(1)) < 0

In [573]:
hh_saaq.loc[saaq_error_driving] = correct_data(hh_saaq.loc[saaq_error_driving], ("hh_driving_license", 4), ("hh_size", 5))
hh_saaq.loc[saaq_error_car] = correct_data(hh_saaq.loc[saaq_error_car], ("hh_car", 4), ("hh_size", 5))

In [574]:
hh_saaq.fillna(value=0, inplace=True)

In [606]:
hh_saaq['hh_car_5'] = hh_saaq.loc[:, 'hh_size_1': 'hh_size_5'].sum(1) - hh_saaq.loc[:, "hh_car_1": "hh_car_4"].sum(1)

hh_saaq.loc[hh_saaq.hh_car_5 < 0, 'hh_car_5'] = 0

In [612]:
hh_saaq['hh_driving_license_5'] = hh_saaq.loc[:, 'hh_size_1': 'hh_size_5'].sum(1) - hh_saaq.loc[:, "hh_driving_license_1": "hh_driving_license_4"].sum(1)

hh_saaq.loc[hh_saaq.hh_driving_license_5 < 0, 'hh_driving_license_5'] = 0

In [614]:
hh_saaq.columns

Index(['DAUID', '1', 'hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4',
       'hh_size_5', 'hh_income_1', 'hh_income_2', 'hh_income_3', 'hh_income_4',
       'hh_income_5', 'hh_type_1', 'hh_type_2', 'geo', 'CTUID', 'new_ct',
       'hh_driving_license_1', 'hh_driving_license_2', 'hh_driving_license_3',
       'hh_driving_license_4', 'hh_car_1', 'hh_car_2', 'hh_car_3', 'hh_car_4',
       'hh_car', 'hh_car_5', 'hh_driving_license_5'],
      dtype='object')

In [615]:
columns = ['geo', 'hh_size_1', 'hh_size_2', 'hh_size_3', 'hh_size_4','hh_size_5', 
           'hh_income_1', 'hh_income_2', 'hh_income_3', 'hh_income_4','hh_income_5',
           'hh_type_1', 'hh_type_2',
           'DAUID', 'CTUID', 'new_ct',
           'hh_driving_license_1', 'hh_driving_license_2', 'hh_driving_license_3','hh_driving_license_4', 
           'hh_driving_license_5', 'hh_car_1', 'hh_car_2', 'hh_car_3', 'hh_car_4','hh_car', 'hh_car_5']

In [616]:
hh_saaq[columns].to_csv("{}/household_marginals.csv".format(path))

## Persons

In [576]:
pp = montreal_da_p_data_new.copy()

pp.rename(columns={'geo':'DAUID'}, inplace=True)
pp['geo'] = pp.index

In [577]:
saaq_data_p = pd.read_csv("../data/quebec_data/person_license_sex.csv")

In [578]:
saaq_data_p

,DAUID,CTUID,new_ct,new_da,license,license_sex_1,license_sex_2
0,24520101,4620690000,657,1,619.0,329.0,290.0
1,24520102,4620691000,658,2,420.0,222.0,198.0
2,24520103,4620691000,658,3,339.0,165.0,174.0
3,24520104,4620691000,658,4,642.0,346.0,296.0
4,24520105,4620691000,658,5,389.0,204.0,185.0
...,...,...,...,...,...,...,...
6436,24750245,4620740030,724,6438,671.0,346.0,325.0
6437,24750247,4620740030,724,6439,1523.0,800.0,723.0
6438,24750248,4620740040,725,6440,2713.0,1410.0,1303.0
6439,24760050,4620740010,723,6441,734.0,413.0,321.0


In [579]:
saaq_data_p

,DAUID,CTUID,new_ct,new_da,license,license_sex_1,license_sex_2
0,24520101,4620690000,657,1,619.0,329.0,290.0
1,24520102,4620691000,658,2,420.0,222.0,198.0
2,24520103,4620691000,658,3,339.0,165.0,174.0
3,24520104,4620691000,658,4,642.0,346.0,296.0
4,24520105,4620691000,658,5,389.0,204.0,185.0
...,...,...,...,...,...,...,...
6436,24750245,4620740030,724,6438,671.0,346.0,325.0
6437,24750247,4620740030,724,6439,1523.0,800.0,723.0
6438,24750248,4620740040,725,6440,2713.0,1410.0,1303.0
6439,24760050,4620740010,723,6441,734.0,413.0,321.0


In [588]:
pp_saaq = pp.merge(saaq_data_p, on='DAUID', how='left')

In [589]:
pp_saaq_errors = (pp_saaq.Pop - pp_saaq['license']) < 0

In [590]:
pp_saaq.loc[pp_saaq_errors]

,DAUID,Pop,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,...,sex_2,workplace_1,workplace_2,geo,CTUID,new_ct,new_da,license,license_sex_1,license_sex_2
64,24560169,5.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,4.0,0.0,5.0,65,4.622010e+09,935.0,65.0,18.0,13.0,5.0
992,24580677,20.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,1.0,...,10.0,0.0,20.0,993,4.620887e+09,886.0,993.0,27.0,20.0,7.0
1165,24590219,608.0,41.0,41.0,21.0,30.0,40.0,56.0,46.0,46.0,...,297.0,10.0,598.0,1166,4.620903e+09,903.0,1166.0,649.0,336.0,313.0
1226,24600077,310.0,10.0,6.0,10.0,10.0,26.0,31.0,20.0,26.0,...,167.0,0.0,310.0,1227,4.620683e+09,627.0,1227.0,345.0,163.0,182.0
1487,24640169,307.0,20.0,26.0,10.0,5.0,15.0,25.0,20.0,15.0,...,139.0,10.0,297.0,1488,4.620689e+09,656.0,1488.0,384.0,208.0,176.0
3390,24661234,395.0,15.0,21.0,20.0,21.0,20.0,15.0,15.0,15.0,...,228.0,20.0,375.0,3391,4.620317e+09,336.0,3391.0,711.0,327.0,384.0
4227,24662144,804.0,15.0,10.0,11.0,20.0,31.0,36.0,26.0,15.0,...,455.0,61.0,743.0,4228,4.620351e+09,361.0,4228.0,863.0,430.0,433.0
5035,24662998,32.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,...,16.0,1.0,31.0,5036,4.620520e+09,452.0,5036.0,470.0,237.0,233.0
5359,24663354,901.0,80.0,79.0,56.0,35.0,45.0,60.0,75.0,85.0,...,491.0,30.0,871.0,5360,4.620421e+09,415.0,5360.0,1288.0,745.0,543.0
5375,24663370,5.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,3.0,0.0,5.0,5376,4.620127e+09,138.0,5376.0,7.0,5.0,5.0


In [591]:
pp_saaq.loc[pp_saaq_errors] = correct_data(pp_saaq.loc[pp_saaq_errors], ('license_sex', 2), ('Pop', 1), tol=5)
pp_saaq['license'] = pp_saaq.loc[:, 'license_sex_1': 'license_sex_2'].sum(1)

pp_saaq.fillna(value=0, inplace=True)

In [598]:
pp_saaq['license_sex_3'] = pp_saaq.Pop - pp_saaq['license']

pp_saaq.loc[pp_saaq.license_sex_3 < 0, 'license_sex_3'] = 0

In [617]:
pp_saaq.columns

Index(['DAUID', 'Pop', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6',
       'age_7', 'age_8', 'age_9', 'age_10', 'age_11', 'age_12', 'age_13',
       'age_14', 'age_15', 'age_16', 'sex_1', 'sex_2', 'workplace_1',
       'workplace_2', 'geo', 'CTUID', 'new_ct', 'new_da', 'license',
       'license_sex_1', 'license_sex_2', 'license_sex_3'],
      dtype='object')

In [619]:
columns = ['geo', 'Pop', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'age_10', 'age_11', 'age_12', 'age_13', 'age_14', 'age_15', 'age_16',
           'sex_1', 'sex_2',
           'workplace_1', 'workplace_2',
           'DAUID', 'CTUID', 'new_ct', 'new_da', 
           'license', 'license_sex_1', 'license_sex_2', 'license_sex_3']

In [620]:
pp_saaq[columns].to_csv("{}/person_marginals.csv".format(path))